In [1]:
import torch
import math

In [2]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

In [3]:
B = 1
N = 128
H = 1
d = 64
query_states = torch.randn(B, H, N, d, device=torch.device("cuda:0"), dtype=torch.bfloat16).requires_grad_()
key_states = torch.randn_like(query_states).requires_grad_()
value_states = torch.randn_like(query_states).requires_grad_()
A = (-torch.nn.functional.softplus(torch.randn(B, N, H, device=torch.device("cuda:0"), dtype=torch.bfloat16))).requires_grad_()

query_states_ = query_states.detach().clone().requires_grad_()
key_states_ = key_states.detach().clone().requires_grad_()
value_states_ = value_states.detach().clone().requires_grad_()
A_ = A.detach().clone().requires_grad_()

In [5]:
order_ = 1
def kron(X):
    return (X[..., None] * X[..., None, :]).flatten(-2, -1)
from Mamba_Custom import mamba_chunk_scan_combined
chunk_size = 64
# Note that mamba 2 still adds the *dt factor to the values. This needs to be removed in the kernel
# It also doesn't have the factor_. To add this, you can just multiply the keys by the factor_
query_states__ = query_states_ * 1/math.sqrt(key_states.shape[-1])
value_states__ = value_states_ * 1/order_
# To get the squared inner product, we need to explicitly do the kron prod
if order_ == 2:
    query_states___ = kron(query_states__)
    key_states__ = kron(key_states_)
else:
    query_states___ = query_states__
    key_states__ = key_states_
check = torch.autograd.gradcheck(mamba_chunk_scan_combined, (value_states__.float().transpose(1, 2).detach().requires_grad_(False), A_.float(), torch.ones(value_states__.shape[1]).float().to(query_states__.device), key_states__.float().transpose(1, 2), query_states___.float().transpose(1, 2), chunk_size, None, None, None, None, None, None, False, (-float("inf"), float("inf")), False, False), eps=1e-4)
print()

/users/gmongaras/miniconda3/lib/python3.10/site-packages/torch/autograd/gradcheck.py:922: UserWarning: Input #1 requires gradient and is not a double precision floating point or complex. This check will likely fail if all the inputs are not of double precision floating point or complex. 
  warnings.warn(
/users/gmongaras/miniconda3/lib/python3.10/site-packages/torch/autograd/gradcheck.py:922: UserWarning: Input #3 requires gradient and is not a double precision floating point or complex. This check will likely fail if all the inputs are not of double precision floating point or complex. 
  warnings.warn(
/users/gmongaras/miniconda3/lib/python3.10/site-packages/torch/autograd/gradcheck.py:922: UserWarning: Input #4 requires gradient and is not a double precision floating point or complex. This check will likely fail if all the inputs are not of double precision floating point or complex. 
  warnings.warn(


GradcheckError: Jacobian mismatch for output 0 with respect to input 0,
numerical:tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., -0.0978, -0.0763, -0.0358],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.0978, -0.0763, -0.0358],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.0978, -0.0763, -0.0358]],
       device='cuda:0')
analytical:tensor([[ 0.7588,  0.7588,  0.7588,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ..., -0.0360, -0.0463, -0.0268],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.4701, -0.4803, -0.4608],
        [ 0.0000,  0.0000,  0.0000,  ..., -0.8468, -1.7886, -1.0932]],
       device='

In [4]:
attention_mask = (torch.tril(torch.ones(N, N)) == 0)[None, None, ...].bool().cuda()
attention_mask

tensor([[[[False,  True,  True,  ...,  True,  True,  True],
          [False, False,  True,  ...,  True,  True,  True],
          [False, False, False,  ...,  True,  True,  True],
          ...,
          [False, False, False,  ..., False,  True,  True],
          [False, False, False,  ..., False, False,  True],
          [False, False, False,  ..., False, False, False]]]], device='cuda:0')

In [5]:
order_ = 1

In [6]:
# Accumulate over orders
factor_ = 1/math.sqrt(key_states.shape[-1])

# Convert to a value between 0 and 2
order_coefficients = 1

# if self.cumulative:
weight_ = 1/order_
_i = order_
attn_weights = (weight_ * (query_states.float() @ key_states.float().mT * factor_)**_i)

# Apply A mask
A_cumsum = torch.cumsum(A.float(), dim=-2).mT
A_mask = (((A_cumsum[:, :, :, None] - A_cumsum[:, :, None, :]))).masked_fill(attention_mask.bool(), -torch.inf).exp().to(query_states.dtype)
attn_weights = attn_weights * A_mask

out = attn_weights.bfloat16() @ value_states

In [7]:
def kron(X):
    return (X[..., None] * X[..., None, :]).flatten(-2, -1)
from Mamba_Custom import mamba_chunk_scan_combined
chunk_size = 256
# Note that mamba 2 still adds the *dt factor to the values. This needs to be removed in the kernel
# It also doesn't have the factor_. To add this, you can just multiply the keys by the factor_
query_states__ = query_states_ * 1/math.sqrt(key_states.shape[-1])
value_states__ = value_states_ * 1/order_
# To get the squared inner product, we need to explicitly do the kron prod
if order_ == 2:
    query_states___ = kron(query_states__)
    key_states__ = kron(key_states_)
else:
    query_states___ = query_states__
    key_states__ = key_states_
out_ = mamba_chunk_scan_combined(x=value_states__.transpose(1, 2), dt=A_, A=torch.ones(value_states__.shape[1]).float().to(query_states__.device), B=key_states__.transpose(1, 2), C=query_states___.transpose(1, 2), chunk_size=chunk_size, D=None, z=None, dt_bias=None, initial_states=None, seq_idx=None, cu_seqlens=None, dt_softplus=False, dt_limit=(-float("inf"), float("inf")), return_final_states=False, return_varlen_states=False).transpose(1, 2)

/users/gmongaras/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
out

tensor([[[[-1.1328e+00, -2.7188e+00,  5.2188e+00,  ...,  1.5547e+00,
           -1.8047e+00, -1.3516e+00],
          [ 6.5918e-02,  2.2852e-01, -1.3770e-01,  ..., -2.4414e-01,
           -1.6699e-01,  3.5400e-02],
          [ 6.9141e-01, -1.1172e+00, -1.6719e+00,  ..., -2.1582e-01,
            4.3750e-01, -1.5859e+00],
          ...,
          [-4.9805e-01, -3.2227e-01, -8.0859e-01,  ..., -8.5938e-01,
           -1.3672e-01, -9.1797e-01],
          [ 2.2031e+00, -1.0938e+00, -2.5156e+00,  ..., -4.9805e-01,
           -1.8516e+00, -1.3984e+00],
          [ 7.1875e-01,  2.2812e+00, -1.7656e+00,  ..., -4.5166e-02,
            3.6094e+00, -3.5938e+00]],

         [[ 3.4766e-01, -2.3804e-02, -5.7031e-01,  ..., -2.3340e-01,
            4.0430e-01,  1.6016e-01],
          [ 6.0938e-01,  4.6289e-01,  2.5781e-01,  ...,  1.4648e-01,
           -4.9219e-01, -7.4219e-01],
          [ 5.6250e-01,  2.9541e-02,  6.9531e-01,  ...,  9.3750e-01,
           -5.5469e-01, -1.1797e+00],
          ...,
     

In [9]:
out_

tensor([[[[-1.1328e+00, -2.7188e+00,  5.2188e+00,  ...,  1.5547e+00,
           -1.8047e+00, -1.3516e+00],
          [ 6.5918e-02,  2.2852e-01, -1.3770e-01,  ..., -2.4414e-01,
           -1.6699e-01,  3.5400e-02],
          [ 6.9141e-01, -1.1172e+00, -1.6719e+00,  ..., -2.1582e-01,
            4.3750e-01, -1.5859e+00],
          ...,
          [-4.9805e-01, -3.2422e-01, -8.0859e-01,  ..., -8.5938e-01,
           -1.3672e-01, -9.1797e-01],
          [ 2.2031e+00, -1.0938e+00, -2.5156e+00,  ..., -5.0000e-01,
           -1.8516e+00, -1.3906e+00],
          [ 7.1484e-01,  2.2969e+00, -1.7578e+00,  ..., -4.6387e-02,
            3.6094e+00, -3.5938e+00]],

         [[ 3.4766e-01, -2.3804e-02, -5.7031e-01,  ..., -2.3340e-01,
            4.0430e-01,  1.6016e-01],
          [ 6.0938e-01,  4.6289e-01,  2.5781e-01,  ...,  1.4648e-01,
           -4.9219e-01, -7.4219e-01],
          [ 5.6250e-01,  2.9663e-02,  6.9531e-01,  ...,  9.3750e-01,
           -5.5469e-01, -1.1797e+00],
          ...,
     

In [10]:
(out - out_).abs().max()

tensor(0.0625, device='cuda:0', dtype=torch.bfloat16, grad_fn=<MaxBackward1>)

In [11]:
out.mean().backward()

In [12]:
out_.mean().backward()

In [13]:
(query_states.grad - query_states_.grad).abs().max()

tensor(3.7253e-09, device='cuda:0', dtype=torch.bfloat16)

In [14]:
(key_states.grad - key_states_.grad).abs().max()

tensor(7.4506e-09, device='cuda:0', dtype=torch.bfloat16)

In [15]:
(value_states.grad - value_states_.grad).abs().max()

tensor(3.7253e-09, device='cuda:0', dtype=torch.bfloat16)

In [16]:
(A.grad - A_.grad).abs().max()

tensor(3.2187e-05, device='cuda:0', dtype=torch.bfloat16)

/users/gmongaras/miniconda3/lib/python3.10/site-packages/torch/autograd/gradcheck.py:922: UserWarning: Input #0 requires gradient and is not a double precision floating point or complex. This check will likely fail if all the inputs are not of double precision floating point or complex. 
  warnings.warn(
/users/gmongaras/miniconda3/lib/python3.10/site-packages/torch/autograd/gradcheck.py:922: UserWarning: Input #1 requires gradient and is not a double precision floating point or complex. This check will likely fail if all the inputs are not of double precision floating point or complex. 
  warnings.warn(
/users/gmongaras/miniconda3/lib/python3.10/site-packages/torch/autograd/gradcheck.py:922: UserWarning: Input #3 requires gradient and is not a double precision floating point or complex. This check will likely fail if all the inputs are not of double precision floating point or complex. 
  warnings.warn(
/users/gmongaras/miniconda3/lib/python3.10/site-packages/torch/autograd/gradcheck.

OutOfMemoryError: CUDA out of memory. Tried to allocate 192.00 MiB. GPU 0 has a total capacity of 79.25 GiB of which 91.38 MiB is free. Process 593366 has 988.00 MiB memory in use. Including non-PyTorch memory, this process has 78.18 GiB memory in use. Of the allocated memory 76.30 GiB is allocated by PyTorch, and 316.50 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [13]:
(query_states.grad - query_states_.grad).abs().max()

tensor(3.7253e-09, device='cuda:0', dtype=torch.bfloat16)

In [14]:
(key_states.grad - key_states_.grad).abs().max()

tensor(7.4506e-09, device='cuda:0', dtype=torch.bfloat16)

In [15]:
(value_states.grad - value_states_.grad).abs().max()

tensor(3.7253e-09, device='cuda:0', dtype=torch.bfloat16)

In [16]:
(A.grad - A_.grad).abs().max()

tensor(3.2187e-05, device='cuda:0', dtype=torch.bfloat16)